In [ ]:
#| default_exp testing

In [ ]:
import sqlite3
import os
import tempfile
from llamaindex_mcp_nbdev.core import SQLiteMCPServer, init_db

In [ ]:
# Test Database Operations
def test_database_operations():
    """Test database initialization and operations."""
    # Use temporary database for testing
    with tempfile.NamedTemporaryFile(suffix='.db', delete=False) as tmp:
        db_path = tmp.name
    
    try:
        # Test database initialization
        conn, cursor = init_db(db_path)
        
        # Test table creation
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='people'")
        tables = cursor.fetchall()
        assert len(tables) == 1, "People table should be created"
        
        # Test data insertion
        cursor.execute("INSERT INTO people (name, age, profession) VALUES ('Test User', 25, 'Tester')")
        conn.commit()
        
        # Test data retrieval
        cursor.execute("SELECT * FROM people")
        results = cursor.fetchall()
        assert len(results) == 1, "Should have one record"
        assert results[0][1] == 'Test User', "Name should match"
        
        conn.close()
        print("Database operations test passed!")
        
    finally:
        # Clean up
        if os.path.exists(db_path):
            os.unlink(db_path)

# Run test
test_database_operations()

In [ ]:
# Test MCP Server
def test_mcp_server():
    """Test MCP server initialization."""
    with tempfile.NamedTemporaryFile(suffix='.db', delete=False) as tmp:
        db_path = tmp.name
    
    try:
        # Test server creation
        server = SQLiteMCPServer('test-server', db_path)
        assert server.db_path == db_path, "Database path should be set"
        assert server.mcp is not None, "MCP instance should be created"
        assert hasattr(server.mcp, 'tool'), "MCP should have tool decorator"
        
        # Test database initialization
        conn, cursor = init_db(db_path)
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='people'")
        tables = cursor.fetchall()
        assert len(tables) == 1, "People table should be created"
        conn.close()
        
        print("MCP server test passed!")
        
    finally:
        # Clean up
        if os.path.exists(db_path):
            os.unlink(db_path)

# Run test
test_mcp_server()